In [1]:
import requests
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
import torch
from torchvision import io

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

image = io.read_image("/Users/andrewmayes/Dev/aimv2-large-patch14-native/coco_cat.jpg")

# processor = AutoImageProcessor.from_pretrained(
#     "model",
#     use_fast=True
# )

model = AutoModel.from_pretrained(
    "model",
    trust_remote_code=True,
)

inputs = {"pixel_values": image.unsqueeze(0).to(dtype=torch.float32)}
# # inputs = processor(images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

In [3]:
outputs.last_hidden_state

tensor([[[-1.3894, -0.4443, -0.8728,  ...,  0.8035,  1.7467,  0.4931],
         [-0.9306, -0.1976, -1.3734,  ...,  0.2957,  1.7883,  0.6244],
         [-0.9875, -0.3535, -0.8236,  ...,  0.3329,  1.9890,  0.6134],
         ...,
         [-2.2533, -1.0144,  0.3371,  ...,  1.8053,  0.6310,  0.2339],
         [-2.9506, -1.5603, -1.7520,  ...,  2.3703,  0.0922, -0.1495],
         [-1.8669, -1.1232, -0.6156,  ...,  0.9939,  0.9958, -0.3018]]])

In [2]:
import numpy as np
tmp2 = torch.tensor(np.load("/Users/andrewmayes/Dev/AIMv2-rs/AimV2_rs/rust_output.npy"))
tmp2

tensor([[[-1.4361, -0.4340, -1.0222,  ...,  0.7725,  1.6183,  0.5495],
         [-1.2150, -0.3754, -1.7095,  ...,  0.2582,  1.6962,  0.5982],
         [-1.2924, -0.5869, -1.1825,  ...,  0.2821,  1.9317,  0.5431],
         ...,
         [-1.2582, -1.1061, -0.9368,  ...,  1.0605,  1.1179,  0.4888],
         [-2.8476, -0.9416, -1.9151,  ...,  1.4845,  0.1211,  0.1786],
         [-1.3360, -0.4403, -1.3850,  ...,  0.6148,  1.3577,  0.1852]]])

In [ ]:
image = io.read_image("/Users/andrewmayes/Dev/aimv2-large-patch14-native/coco_cat.jpg")
image.size()

torch.Size([3, 480, 640])

In [5]:
numpy_array = image.cpu().numpy()
np.save("tmp.npy", numpy_array)
print("Saved tensor to tmp.npy")

Saved tensor to tmp.npy


In [10]:
torch.save(image, "tmp.pt")

In [9]:
np.save("tmp.npy", image.numpy())

In [ ]:
import torch
import numpy as np
tmp2 = torch.tensor(np.load("/Users/andrewmayes/Dev/AIMv2-rs/AimV2_rs/image.npy"), dtype=torch.uint8).squeeze()
tmp2.size()

torch.Size([3, 480, 640])

In [13]:
for idx, (val1, val2) in enumerate(zip(tmp2.flatten(), outputs.last_hidden_state.flatten())):
    if val1 != val2:
        print(idx, val1.item(), val2.item(), abs(val1.item() - val2.item()))

0 -1.4361271858215332 -1.3894448280334473 0.04668235778808594
1 -0.4339924156665802 -0.44434893131256104 0.010356515645980835
2 -1.022186517715454 -0.8728010654449463 0.1493854522705078
3 1.4050524234771729 1.3757328987121582 0.02931952476501465
4 0.4454251825809479 0.6461723446846008 0.20074716210365295
5 -0.1804673671722412 -0.06661732494831085 0.11385004222393036
6 2.6255545616149902 2.9444756507873535 0.3189210891723633
7 0.6327552199363708 0.525600790977478 0.10715442895889282
8 -0.9358406066894531 -0.9042074680328369 0.03163313865661621
9 -1.853401780128479 -2.0134196281433105 0.16001784801483154
10 -1.6637554168701172 -1.6906921863555908 0.026936769485473633
11 0.5644355416297913 0.6031080484390259 0.03867250680923462
12 -2.7667994499206543 -2.4014434814453125 0.3653559684753418
13 3.7448267936706543 3.479053020477295 0.2657737731933594
14 2.444406270980835 2.2139384746551514 0.2304677963256836
15 1.279447078704834 1.4355500936508179 0.1561030149459839
16 -1.4087302684783936 -1.

KeyboardInterrupt: 

In [6]:
(tmp2 == image).sum()

tensor(906133)

In [7]:
3 * 480 * 640

921600

In [ ]:
torch.allclose(image, tmp2)

False

In [ ]:
import requests
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
import torch
import numpy as np # Often useful for examining differences

# --- Configuration ---
# IMPORTANT: Replace "model" with an actual model identifier from Hugging Face Hub
# Examples: "google/vit-base-patch16-224-in21k", "facebook/dinov2-base", "microsoft/beit-base-patch16-224-pt22k-ft22k"
MODEL_NAME = "model" # <--- CHANGE THIS
# ---             ---

# --- Load Image ---
url = "/Users/andrewmayes/Dev/aimv2-large-patch14-native/coco_cat.jpg"
try:
    image = Image.open(requests.get(url, stream=True).raw).convert("RGB") # Ensure RGB
    print(f"Image loaded successfully from {url}")
except Exception as e:
    print(f"Error loading image: {e}")
    exit()
# ---           ---

# --- Load Processors ---
print(f"Loading processors for model: {MODEL_NAME}")
try:
    # Note: For many models, use_fast=True/False might not change behavior significantly
    # for the *image processor* itself (it's more impactful for tokenizers).
    # However, it's good practice to test if you suspect differences.
    p1 = AutoImageProcessor.from_pretrained(
        MODEL_NAME,
        use_fast=True # Often implied or doesn't exist for image proc.
                        # Let's stick to default or explicitly False for comparison base
    )

    p2 = AutoImageProcessor.from_pretrained(
        MODEL_NAME,
        # use_fast=False # Explicitly set to False if you want to be sure
                          # But often the default *is* the 'slow' Python version
    )
    print("Processors loaded.")

    # Optional: Check if they are actually different classes (unlikely for images)
    print(f"Processor 1 type: {type(p1)}")
    print(f"Processor 2 type: {type(p2)}")
    # If the types are the same, use_fast likely had no effect for this specific processor.

except Exception as e:
    print(f"Error loading processors for model '{MODEL_NAME}': {e}")
    print("Please ensure the model name is correct and exists on the Hugging Face Hub.")
    exit()
# ---               ---


# --- Process Image ---
print("Processing image with both processors...")
i1 = p1(images=image, return_tensors="pt")
i2 = p2(images=image, return_tensors="pt")
print("Image processing complete.")
# ---               ---

# --- Compare Outputs ---
print("\n--- Comparing Pixel Values ---")

# Check if 'pixel_values' key exists
if "pixel_values" not in i1 or "pixel_values" not in i2:
    print("Error: 'pixel_values' key not found in processor output.")
    print("Output keys p1:", i1.keys())
    print("Output keys p2:", i2.keys())
    exit()

t1 = i1["pixel_values"]
t2 = i2["pixel_values"]

# 1. Check Basic Properties
print(f"Shape t1: {t1.shape}, Dtype t1: {t1.dtype}")
print(f"Shape t2: {t2.shape}, Dtype t2: {t2.dtype}")
if t1.shape != t2.shape or t1.dtype != t2.dtype:
    print("🛑 Basic properties (shape/dtype) differ!")
else:
    print("✅ Basic properties (shape/dtype) match.")

# 2. Check for Exact Equality (less reliable for floats)
all_equal_direct = torch.equal(t1, t2) # torch.equal checks for exact match
print(f"\nExact element-wise equality (torch.equal): {all_equal_direct}")

# 3. Check for Near Equality (Recommended)
# Uses default tolerances (rtol=1e-05, atol=1e-08) which are usually good.
all_close = torch.allclose(t1, t2,rtol = 0.00001, atol = 1e-8,)
print(f"Near element-wise equality (torch.allclose, default tol): {all_close}")

# 4. Quantify the Difference (if they are not close)
if not all_close:
    print("\nInvestigating differences:")
    difference = t1 - t2
    abs_difference = torch.abs(difference)

    print(f"  Max absolute difference: {torch.max(abs_difference).item():.6e}")
    print(f"  Mean absolute difference: {torch.mean(abs_difference).item():.6e}")
    # Optional: Check where the max difference occurs
    # max_diff_index = torch.argmax(abs_difference)
    # print(f"  Index of max difference: {np.unravel_index(max_diff_index.item(), t1.shape)}")
    # print(f"  Value t1 at max diff: {t1.flatten()[max_diff_index].item():.6f}")
    # print(f"  Value t2 at max diff: {t2.flatten()[max_diff_index].item():.6f}")
else:
     print("✅ Tensors are numerically close.")

print("\n--- Comparison Finished ---")

# Optional: Pass through model (uncomment if needed)
# print("\n--- Running Model (Optional) ---")
# try:
#     model = AutoModel.from_pretrained(
#         MODEL_NAME,
#         trust_remote_code=True, # Be cautious with this flag
#     )
#     print("Model loaded.")
#     with torch.no_grad(): # Disable gradient calculation for inference
#         outputs1 = model(**i1)
#         outputs2 = model(**i2)
#     print("Model inference complete.")
#     # You could then compare outputs1 and outputs2 similarly
#     # e.g., compare outputs1.last_hidden_state and outputs2.last_hidden_state
#     if hasattr(outputs1, 'last_hidden_state') and hasattr(outputs2, 'last_hidden_state'):
#        lhs_close = torch.allclose(outputs1.last_hidden_state, outputs2.last_hidden_state)
#        print(f"Model last_hidden_state close: {lhs_close}")
#        if not lhs_close:
#             diff_lhs = torch.abs(outputs1.last_hidden_state - outputs2.last_hidden_state)
#             print(f"  Max abs diff (last_hidden_state): {torch.max(diff_lhs).item():.6e}")
#             print(f"  Mean abs diff (last_hidden_state): {torch.mean(diff_lhs).item():.6e}")
#
# except Exception as e:
#      print(f"Error loading or running model '{MODEL_NAME}': {e}")

Image loaded successfully from http://images.cocodataset.org/val2017/000000039769.jpg
Loading processors for model: model
Processors loaded.
Processor 1 type: <class 'transformers.models.clip.image_processing_clip_fast.CLIPImageProcessorFast'>
Processor 2 type: <class 'transformers.models.clip.image_processing_clip.CLIPImageProcessor'>
Processing image with both processors...
Image processing complete.

--- Comparing Pixel Values ---
Shape t1: torch.Size([1, 3, 480, 640]), Dtype t1: torch.float32
Shape t2: torch.Size([1, 3, 480, 640]), Dtype t2: torch.float32
✅ Basic properties (shape/dtype) match.

Exact element-wise equality (torch.equal): False
Near element-wise equality (torch.allclose, default tol): False

Investigating differences:
  Max absolute difference: 2.384186e-07
  Mean absolute difference: 5.518056e-08

--- Comparison Finished ---


In [2]:
i1["pixel_values"] == i2["pixel_values"]

tensor([[[[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [ True,  True,  True,  ..., False, False, False],
          [False, False,  True,  ..., False, False, False],
          [ True,  True,  True,  ..., False, False, False]],

         [[False, False,  True,  ...,  True,  True,  True],
          [False, False,  True,  ..., False,  True,  True],
          [False, False,  True,  ...,  True,  True,  True],
          ...,
          [False, False, False,  ..., False,  True,  True],
          [False,  True, False,  ...,  True,  True,  True],
          [False, False, False,  ...,  True, False,  True]],

         [[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False,  True,  True,  ..., False, False,  True],
          ...,
          [ True,  True,  True,  ..., False, False,